# lambda介紹

什麼是lambda？
lambda是一種希臘字母，本身並沒有意義。


開玩笑的，lambda，準確來說lambda-calculus，是一種函數的表達方式。
以下是python底下的lambda表達方式。

In [1]:
lambda x: x + 1
# 輸入參數 x，返回 x + 1

<function __main__.<lambda>(x)>

在python裡，單單上面的表達式無法作為函數使用，需要「調用」。

In [2]:
f = lambda x: x + 1
print(f(3))

4


well... 這其實相當於以下函數定義。

In [5]:
def f(x):
    return x + 1

print(f(3))

4


不同於函數定義，lambda表達式可以直接作為函數參數傳遞。

In [14]:
def do_function(func, *params):
    return func(*params)

print(do_function(lambda x: x+1, 3))

4


當然，lambda可以傳給另一個lambda。

In [10]:
print((lambda x,*y: x(*y))(lambda x: x+1, 3))

4


或是讓lambda回傳一個lambda

In [ ]:
a = (lambda x: (lambda y: x+y))(3)
# a = lambda y: 3+y
print(a(5))

8


## 進入lambda領域

在踏入完全地lambda領域之前，你需要知道三個概念。
1. 變數 a

    任何一個變數都是一個有效的表達式，可以是任何字元。

2. lambda函數體 lambda a:b

    接受參數a並回傳b。非程式語言會以希臘字母和句號紀錄: λa.b

3. 傳遞參數 lambda a:b (c) / (λa.b)(c)

    將傳入值c代進函數體b中所有a的地方
    也有人利用(_ _)代表套用函數，即 (λa.b c)
    但後者較難以追蹤函數體與傳入值間的關係與差異

## alpha 恆等

當表達式彼此間只差在變數名稱的不同，其餘結構相等時，
稱兩表達式彼此alpha 恆等。

λa.a <-> λb.b

In [ ]:
print((lambda x: x) == (lambda y: y)) 

False


well... 即便python不這麼認為w

## beta 簡化
當一表達式經過 beta 簡化，可以與另一表達式 alpha 恆等，則稱為 beta 恆等。

(λa.a)(b) <-> b

In [21]:
print((lambda x: x)(3) == 3)

True


## lambda 簡化
實質上，lambda 只接受一個變數，
但是只要讓一個 lambda 函數回傳另一個 lambda 函數，
即可實現多參數傳遞，
因此也可以將多個單參數 lambda 簡化合併。

(λa.λb.a+b)(1)(2) <-> (λb.1+b)(2) <-> 1+2

因此
(λa.λb.a+b)(1)(2) <-> (λab.a+b)(1, 2)

在 python 中，後者較為常見。

In [25]:
print((lambda x: (lambda y: x + y))(3)(5))
print((lambda x, y: x + y)(3, 5))

8
8


值得注意的是，函數的帶入方式不一樣，

前者是將 3 代入函數後，先回傳一新函數 λy.3+y，
再將 5 代入 λy.3+y

後者是同時將 3, 5 作為參數 x, y 傳入。

雖然簡化後意義改變，但仍然是 beta 恆等的。

## 布林值
布林值聽著可怕、單身又醜陋，但其實只是邏輯中的基本概念「真」、「假」/「是」、「否」。

雖然很突然，但我想跟你說，這兩位其實是個函數。

真 <-> λab.a
假 <-> λab.b

即：照順序代入 a, b後，返回 a 就是真、b 就是假

為什麼？因為希望 if A then B else C 表達式中可以完美運作。
以下帶你建構上述表達式。

我們希望「真」時返回 B，「假」時返回 C。
因此需要三個參數同時傳入：

(λxyz.?)(λab.a)(B)(C) <-> B

(λxyz.?)(λab.b)(B)(C) <-> C

由於「真」、「假」同時具有選擇回傳值的特性，
讓布林值選擇即可，
因此：

(λxyz.(x)(y, z))(λab.a, B, C)
<-> (λab.a)(B, C)
<-> B

(λxyz.(x)(y, z))(λab.b, B, C)
<-> (λab.b)(B, C)
<-> C

In [ ]:
true = lambda a, b: a
false = lambda a, b: b

print((lambda x, y, z: x(y, z))(true, 1, 2))
print((lambda x, y, z: x(y, z))(false, 1, 2))

1
2
